In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import openpyxl
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import os

# Read the HTML file
with open('Amazon.html', 'r', encoding='utf-8') as file:
    soup = BeautifulSoup(file, 'html.parser')

# Grab each product container
divs = soup.find_all('div', {'data-component-type': 's-search-result'})

products = []
prices = []

for div in divs:
    # --- Product Name ---
    title_tag = div.select_one("div[data-cy='title-recipe'] a")
    product_name = title_tag.get_text(strip=True) if title_tag else ""

    # --- Product Price ---
    whole = div.select_one("span.a-price-whole")
    fraction = div.select_one("span.a-price-fraction")
    if whole:
        product_price = whole.get_text(strip=True) + (fraction.get_text(strip=True) if fraction else "")
    else:
        product_price = ""

    # ✅ Filter: Only keep items where "ROSIER" is in product name
    if "ROSIER" in product_name.upper():
        products.append(product_name)
        prices.append(product_price)

# DataFrame बनाओ
df = pd.DataFrame({
    'Product_Name': products,
    'Product_Price': prices
})

df.to_excel('amazon_products.xlsx', index=False)

print("Scraping complete. Saved amazon_products.xlsx")

# Save to Excel
excel_file = 'amazon_products.xlsx'
workbook = openpyxl.Workbook()
sheet = workbook.active
sheet.title = "Ghee Products"

# Email setup
email_address = "Priyanshkhandelwal946215@gmail.com"
password = "iksg bgeq ylly gyyz"  # Replace with your app-specific password

# Create email
msg = MIMEMultipart()
msg['From'] = email_address
msg['To'] = email_address
msg['Subject'] = 'Amazon Product Data'

# Attach Excel file
with open(excel_file, 'rb') as f:
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(f.read())
part.add_header('Content-Disposition', f'attachment; filename={excel_file}')
encoders.encode_base64(part)
msg.attach(part)

# Send email
try:
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(email_address, password)
    server.sendmail(email_address, email_address, msg.as_string())
    server.quit()
    print("Email sent successfully")
except Exception as e:
    print(f"Error sending email: {e}")

Scraping complete. Saved amazon_products.xlsx
Email sent successfully
